# Libraries and modules

In [50]:
!pip install geopandas

import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import descartes
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point, Polygon
from functools import partial
import pyproj
from pyproj import Transformer
from pyproj import CRS
from shapely.ops import transform

#pd.set_option('display.max_columns', 500)
#pd.set_option('display.max_rows', 500)

# Functions

In [51]:
def gen_points_inside_polygon(polygon, miles=1):
  '''
  Takes polygon and miles, creates grid around edges and iterates in .25 mile 
  steps around grid, returning all points within
  '''

  lat_increment = miles * 0.0145
  lon_increment = miles * 0.02 #this is tied to latitude
  
  #four corners of a grid
  min_lat, min_lon, max_lat, max_lon, = polygon.bounds

  # lists the get appended as grid gets iterated over
  lons, lats = [],[]
  
  lat = min_lat + (lat_increment /2)
  while lat < max_lat:
    lats.append(lat)
    lat+=lat_increment
  lats.append(max_lat)

  #repeat for long; dif converstion
  lon = min_lon + (lon_increment / 2)
  while lon < max_lon:
    lons.append(lon)
    lon+= lon_increment
  lons.append(max_lon)

  #iterate through all lat/lon points, instantiate shapely point objs, append to list
  points = []
  for i in range(len(lons)):
    for j in range(len(lats)):
      points.append(Point(lons[i], lats[j]))
      #points.append(Point(lats[j], lons[i]))

  points_inside_polygon = []
  for i in range(len(points)):
    if polygon.contains(points[i]):
      points_inside_polygon.append(points[i])

  # if no points, add center point
  if len(points_inside_polygon) == 0:
    points_inside_polygon.append(polygon.centroid)

  return points_inside_polygon

# Make circle
def make_circle(lat, lon, miles=1): #.25-mile in urban areas and 10-mile in rural
  km = miles * 1.60934
  proj_wgs84 = pyproj.CRS("WGS84")
  # Azimuthal equidistant projection
  aeqd_proj = '+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0'
  project = partial(
    pyproj.transformer.transform,
    pyproj.Proj(aeqd_proj.format(lat=lat, lon=lon)),
    proj_wgs84)
  buff = Point(0, 0).buffer(km * 1000)
  return Polygon(transform(project, buff).exterior.coords[:])

def get_num_groche_in_circle(df, circle):
  min_lat, min_lon, max_lat, max_lon = circle.bounds
  df = df[(df.LATITUDE >= min_lat)&(df.LATITUDE <= max_lat)&(df.LONGITUDE>=min_lon)&(df.LONGITUDE<=max_lon)]
  num_groche = 0

  for i in range(len(df)):
    if circle.contains(df.iloc[i]['LOCATION']):
      num_groche+=1

  return num_groche

def desert_search(zip_df, markets_df):
  return_dict = {}

  for i in range(len(zip_df)):
    zip_code = zip_df.iloc[i]['zip_code']
    #zip_code = zip_df.loc[i,'zip_code']
    nearby_groche_count = []
    desert_list = []

    # now iterate over test_points per zip
    for point in zip_df.iloc[i]['test_points']:
      #lat, lon = point.y, point.x
      lon, lat = point.x, point.y
      #make circle around points
      circle = make_circle(lat,lon)
      #get num of groche within circle and add to count for zip
      num_nearby = get_num_groche_in_circle(markets_df, circle)
      nearby_groche_count.append(num_nearby)
      #if there are none, add point to list of food deserts
      if num_nearby==0:
        desert_list.append(point)
    
    num_food_deserts = len(desert_list)

    if zip_code in return_dict:
      return_dict[zip_code]['nearby_groche_count'] += nearby_groche_count
      return_dict[zip_code]['desert_list'] += desert_list
      return_dict[zip_code]['num_food_deserts'] += num_food_deserts
    else:
      return_dict[zip_code]={'nearby_groche_count': nearby_groche_count,
                             'desert_list': desert_list,
                             'num_food_deserts': num_food_deserts}

  #return master dict
  return return_dict

def unpack_desert_search(desert_search_results):
  #initialiaze
  unpacked = {'zip_code': [], 'nearby_groche_count': [], 'desert_list': [], 'num_food_deserts': []}

  #get zip code
  for k, v in desert_search_results.items():
    unpacked['zip_code'].append(k)
    #now get other key va pairs
    for key, val in v.items():
      unpacked[key].append(val)
  unpacked_df = pd.DataFrame(unpacked)

  desert_list=[]
  for i in range(len(unpacked_df)):
    deserts = unpacked_df.iloc[i].desert_list
    if len(deserts)>0:
      for desert in deserts:
        desert_list.append(desert)

  unpacked_df = unpacked_df.drop(columns=['desert_list'])

  def get_avg(num_list):
    return round(sum(num_list)/len(num_list),1)

  unpacked_df.nearby_groche_count = unpacked_df.nearby_groche_count.apply(get_avg)
  unpacked_df = unpacked_df.rename(columns={'nearby_groche_count':'avg_nearby_groche_counts'})

  return unpacked_df, desert_list

# Data Load

In [52]:


#polygon related
polygon_file = '/content/drive/MyDrive/DATA 606 Capstone Project/Datasets and Information/CSVs/zip_shape/tl_2021_us_zcta520.shp'
polygons = gpd.read_file(polygon_file)
zip_df = polygons[['ZCTA5CE20', 'geometry']].rename(columns={'ZCTA5CE20':'zip_code'})
zip_df['zip_code'] = zip_df['zip_code'].astype(int)

#zip_df.to_file('/content/drive/MyDrive/DATA 606 Capstone Project/Datasets and Information/CSVs/cleaned_zip_geodata_shapefile.shp',driver='ESRI Shapefile')



In [53]:
df = pd.read_csv('/content/drive/MyDrive/DATA 606 Capstone Project/Datasets and Information/CSVs/SNAP_Data.csv')

metros = ['Atlanta','Baltimore','DC','Hartford','Minneapolis','Phoenix']
years = "199|198|197"
exclude_string = "Unknown|Delivery Route|Convenience Store|Combination Grocery/Other|Bakery Specialty|Meat/Poultry Specialty|Fruits/Veg Specialty|Seafood Specialty|Super Store|Wholesaler|Military Commissary"

df = df[~df['Store Type'].str.contains(exclude_string, flags=re.IGNORECASE, regex=True) == True].reset_index(drop=True)
df = df[~df['End Date'].str.contains(years, flags=re.IGNORECASE, regex=True) == True].reset_index(drop=True)
df = df.rename(columns={'Zip Code':'zip_code', 'Longitude':'LONGITUDE', 'Latitude':'LATITUDE'})
df['LOCATION']='one'

In [54]:
# for 1 mile graph only
full_desert = pd.DataFrame(columns=['zip_code', 'geometry', 'avg_nearby_groche_counts', 'num_food_deserts'])
for metro in metros:
  metro_zip = pd.read_csv('/content/drive/MyDrive/DATA 606 Capstone Project/Datasets and Information/CSVs/'+metro+'/'+metro+'_county_zips.csv')
  markets = df[df['Metro']==metro].reset_index(drop=True)
  print(len(metro))


  for i in range(len(markets)):
    lat = markets.at[i, 'LATITUDE']
    lon = markets.at[i, 'LONGITUDE']
    markets.at[i, 'LOCATION'] = Point(lat, lon)

  metro_zip_list = metro_zip['zip'].to_list()
  metro_zip_df = zip_df[zip_df['zip_code'].isin(metro_zip_list)]

  metro_zip_df['test_points'] = metro_zip_df.geometry.apply(gen_points_inside_polygon)

  metro_zip_df['num_points'] = metro_zip_df['test_points'].apply(len)

  metro_search = desert_search(metro_zip_df, markets)
  zip_code_groche_df, all_deserts = unpack_desert_search(metro_search)

  #plotting tasks
  polygons_df = metro_zip_df[['zip_code', 'geometry']]
  #convert list from grid search to df
  geo = {'geometry': all_deserts}
  deserts_gdf = gpd.GeoDataFrame(geo)

  groche_density_df = polygons_df.merge(zip_code_groche_df, left_on='zip_code',right_on='zip_code',how='outer')
  full_desert = full_desert.append(groche_density_df)
#full_desert.to_csv('/content/drive/MyDrive/DATA 606 Capstone Project/Datasets and Information/CSVs/Groche Density/all_metro_mile_groche_density.csv')

7


/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


9


/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


2


...

/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


11


/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


7


/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [55]:
groche_density_df.columns

Index(['zip_code', 'geometry', 'avg_nearby_groche_counts', 'num_food_deserts'], dtype='object')

In [56]:
full_desert

,zip_code,geometry,avg_nearby_groche_counts,num_food_deserts
0,30309,"POLYGON ((-84.40785 33.79728, -84.40784 33.798...",6.0,0
1,30518,"POLYGON ((-84.10169 34.11238, -84.10159 34.113...",0.0,1
2,30334,"POLYGON ((-84.39091 33.74693, -84.39021 33.747...",23.0,0
3,30331,"POLYGON ((-84.66326 33.65653, -84.66311 33.656...",0.0,1
4,30319,"POLYGON ((-84.35928 33.88596, -84.35921 33.886...",1.0,0
...,...,...,...,...
167,85122,"POLYGON ((-111.82675 32.95262, -111.82668 32.9...",0.0,1
168,85388,"POLYGON ((-112.47009 33.58361, -112.47008 33.5...",0.0,1
169,85173,"POLYGON ((-111.30067 33.25802, -111.29735 33.2...",0.0,1
170,85375,"POLYGON ((-112.40401 33.68047, -112.40385 33.6...",0.0,1


## Rural

In [ ]:
# Make circle for Rural at 10 miles
def make_circle(lat, lon, miles=10): #1-mile in urban areas and 10-mile in rural
  km = miles * 1.60934
  proj_wgs84 = pyproj.CRS("WGS84")
  # Azimuthal equidistant projection
  aeqd_proj = '+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0'
  project = partial(
    pyproj.transformer.transform,
    pyproj.Proj(aeqd_proj.format(lat=lat, lon=lon)),
    proj_wgs84)
  buff = Point(0, 0).buffer(km * 1000)
  return Polygon(transform(project, buff).exterior.coords[:])

# rural is 10 miles
for metro in metros:
  metro_zip = pd.read_csv('/content/drive/MyDrive/DATA 606 Capstone Project/Datasets and Information/CSVs/'+metro+'/'+metro+'_county_zips.csv')
  markets = df[df['Metro']==metro].reset_index(drop=True)
  print(len(metro))


  for i in range(len(markets)):
    lat = markets.at[i, 'LATITUDE']
    lon = markets.at[i, 'LONGITUDE']
    markets.at[i, 'LOCATION'] = Point(lat, lon)

  metro_zip_list = metro_zip['zip'].to_list()
  metro_zip_df = zip_df[zip_df['zip_code'].isin(metro_zip_list)]

  metro_zip_df['test_points'] = metro_zip_df.geometry.apply(gen_points_inside_polygon)

  metro_zip_df['num_points'] = metro_zip_df['test_points'].apply(len)

  metro_search = desert_search(metro_zip_df, markets)
  zip_code_groche_df, all_deserts = unpack_desert_search(metro_search)

  #plotting tasks
  polygons_df = metro_zip_df[['zip_code', 'geometry']]
  #convert list from grid search to df
  geo = {'geometry': all_deserts}
  deserts_gdf = gpd.GeoDataFrame(geo)

  groche_density_df = polygons_df.merge(zip_code_groche_df, left_on='zip_code',right_on='zip_code',how='outer')
  groche_density_df.to_csv('/content/drive/MyDrive/DATA 606 Capstone Project/Datasets and Information/CSVs/Groche Density/{}_mile_groche_density.csv'.format(metro))
  """
  ax = polygons_df.plot(figsize=(20, 20), color='#00cc66', edgecolor='white')
  deserts_gdf.plot(color='#00004d', alpha=0.05, ax=ax)
  ax.set_facecolor('#f0f0f5')
  plt.grid(linewidth=0.2)
  plt.title('Food Deserts of '+metro+' Metro Area', fontsize=20)
  #plt.figtext(.5,.9,'Food Deserts of Los Angeles County', fontsize=20, ha='center')
  plt.xlabel('Longitude', fontsize=15)
  plt.ylabel('Latitude', fontsize=15)
  """

  # GRAB THESE BOTTOM LINES FOR GRAPHING
  fig, ax = plt.subplots(figsize = (25,25))
  cmap = 'Spectral_r'
  #Color bar is created below
  sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=0, vmax=1))
  # Empty array for the data range
  sm._A = []
  # Add the colorbar to the figure
  cbar = fig.colorbar(sm, fraction=.04)

  ax.set_facecolor('#f0f0f5')
  plt.title('Grocery Store Density (Per 10-mile Radius) for the '+metro+' Metro Area [Rural]',fontsize=20)
  plt.xlabel('Longitude', fontsize=15)
  plt.ylabel('Latitude',fontsize=15)
  plt.grid(linewidth=0.2)
  #ax.set_ylim([38,40])
  #ax.set_xlim([-79,-76])
  ax_zip = groche_density_df.plot(column='avg_nearby_groche_counts', ax=ax, cmap=cmap, vmin=0, vmax =20)
  fig.savefig('/content/drive/MyDrive/DATA 606 Capstone Project/Datasets and Information/Graphs/{}_urban_desert.png'.format(metro))

7


/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


## Urban

In [ ]:
# Make circle for Urban at .5 miles
def make_circle(lat, lon, miles=.5): #1-mile in urban areas and 10-mile in rural
  km = miles * 1.60934
  proj_wgs84 = pyproj.CRS("WGS84")
  # Azimuthal equidistant projection
  aeqd_proj = '+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0'
  project = partial(
    pyproj.transformer.transform,
    pyproj.Proj(aeqd_proj.format(lat=lat, lon=lon)),
    proj_wgs84)
  buff = Point(0, 0).buffer(km * 1000)
  return Polygon(transform(project, buff).exterior.coords[:])


# For Urban  areas miles = .5
for metro in metros:
  metro_zip = pd.read_csv('/content/drive/MyDrive/DATA 606 Capstone Project/Datasets and Information/CSVs/'+metro+'/'+metro+'_county_zips.csv')
  markets = df[df['Metro']==metro].reset_index(drop=True)
  print(len(metro))


  for i in range(len(markets)):
    lat = markets.at[i, 'LATITUDE']
    lon = markets.at[i, 'LONGITUDE']
    markets.at[i, 'LOCATION'] = Point(lat, lon)

  metro_zip_list = metro_zip['zip'].to_list()
  metro_zip_df = zip_df[zip_df['zip_code'].isin(metro_zip_list)]

  metro_zip_df['test_points'] = metro_zip_df.geometry.apply(gen_points_inside_polygon)

  metro_zip_df['num_points'] = metro_zip_df['test_points'].apply(len)

  metro_search = desert_search(metro_zip_df, markets)
  zip_code_groche_df, all_deserts = unpack_desert_search(metro_search)

  #plotting tasks
  polygons_df = metro_zip_df[['zip_code', 'geometry']]
  #convert list from grid search to df
  geo = {'geometry': all_deserts}
  deserts_gdf = gpd.GeoDataFrame(geo)

  groche_density_df = polygons_df.merge(zip_code_groche_df, left_on='zip_code',right_on='zip_code',how='outer')
  groche_density_df.to_csv('/content/drive/MyDrive/DATA 606 Capstone Project/Datasets and Information/CSVs/Groche Density/{}_urban_groche_density.csv'.format(metro))


  # GRAB THESE BOTTOM LINES FOR GRAPHING
  fig, ax = plt.subplots(figsize = (25,25))
  cmap = 'Spectral_r'
  #Color bar is created below
  sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=0, vmax=1))
  # Empty array for the data range
  sm._A = []
  # Add the colorbar to the figure
  cbar = fig.colorbar(sm, fraction=.04)

  ax.set_facecolor('#f0f0f5')
  plt.title('Grocery Store Density (Per 0.5-mile Radius) for the '+metro+' Metro Area [Urban]',fontsize=20)
  plt.xlabel('Longitude', fontsize=15)
  plt.ylabel('Latitude',fontsize=15)
  plt.grid(linewidth=0.2)
  #ax.set_ylim([38,40])
  #ax.set_xlim([-79,-76])
  ax_zip = groche_density_df.plot(column='avg_nearby_groche_counts', ax=ax, cmap=cmap, vmin=0, vmax =10)
  fig.savefig('/content/drive/MyDrive/DATA 606 Capstone Project/Datasets and Information/Graphs/{}_urban_desert.png'.format(metro))